# 1 Data Download

In [5]:
# Download Dataset
import requests
import zipfile
from pathlib import Path
import gdown

# Setup paths and folders names and urls
data_path = Path("data/")
image_path = data_path / "resized/"
dataset_url = 'https://drive.google.com/uc?id=1ve2fCainStQAwTtbdvtX1vRVNJdbcxVi'
train_url = 'https://drive.google.com/uc?id=1uDYbmu12_SykkUub0OVKCywNR9614M-g'


# If the image folder doesn't exist, download it
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    data_path.mkdir(parents=True, exist_ok=True)

    print("Downloading train.csv...")
    gdown.download(train_url, str(data_path / "rotation_angles.csv"), quiet=False)

    print("Downloading Dataset...")
    zip_path = str(data_path / "resized.zip")
    gdown.download(dataset_url, zip_path, quiet=False)

    # Unzip data
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        print("Unzipping dataset data...")
        zip_ref.extractall(image_path)

data/resized directory exists.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 2 Data Import

In [19]:
import pandas as pd
import numpy as np
import io
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import transforms
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image

In [20]:
# Path to the image directory and CSV file
train_dir = "./data/resized"
csv_file = "./data/train.csv"

# Reading the CSV file using Pandas
data = pd.read_csv(csv_file, names=["Image", "Angle"], header=0)
data.head()

,Image,Angle
0,w_1.jpg,-79.58
1,w_100.jpg,-166.04
2,w_1000.jpg,6.89
3,w_1003.jpg,55.08
4,w_1004.jpg,-267.50


In [21]:
data_clean = data[data['Image'].isin(os.listdir(train_dir))]
data_clean.describe()

,Angle
count,4543.000000
mean,-102.097781
std,104.234210
min,-270.000000
25%,-192.655000
50%,-106.190000
75%,-18.095000
max,89.700000


## 3 Dataset and Initialisation

In [32]:
NUM_CLASSES = 360

# Define the Dataset class
class RotationDataset:
    def __init__(self, dataframe, img_folder, transform=None):
        self.dataframe = dataframe
        self.img_folder = img_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_name = row['Image']
        angle = row['Angle']

        try:
            img_path = os.path.join(self.img_folder, image_name)
            img = Image.open(img_path).convert('RGB')

            if self.transform:
                img = self.transform(img)
            return img, angle
        except Exception as e:
            print(e)
            print(angle, image_name)

class AngleEstimationResNet50(nn.Module):
    def __init__(self, num_classes=360):
        super(AngleEstimationResNet50, self).__init__()

        # Load the pre-trained ResNet50 model
        resnet50 = models.resnet50(pretrained=True)

        # Remove the fully connected layers at the end
        self.features = nn.Sequential(*list(resnet50.children())[:-2])

        # Add custom fully connected layers for angle estimation
        self.fc1 = nn.Linear(2048, 512)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.mean(x, dim=(2, 3))  # Global average pooling
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [33]:
BATCH_SIZE = 24

# Set random seed for reproducibility
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the dataset and split it into training and validation sets
train_data, valid_data = train_test_split(data_clean, test_size=0.2, random_state=42)

# Create dataset and dataloaders
train_dataset = RotationDataset(train_data, train_dir, transform)
valid_dataset = RotationDataset(valid_data, train_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model, loss function, and optimizer
model = AngleEstimationResNet50()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Train Model

In [34]:
NUM_EPOCHS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(model, train_loader, criterion, optimizer, device, num_epochs=NUM_EPOCHS):
    for epoch in range(num_epochs):
        # Training
        model.train()
        model.to(device)
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
             running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Train the model
train(model, train_loader, criterion, optimizer, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 360])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [ ]:
throw Exception

### Evaluation

In [ ]:
inputs